Weak perturbation
=================

In this numerical experiment, one considers the propagation of a small perturbation in a collisionless plasma model. The perturbation is initiated by a thin slab of fluid slowly moving with respect to the surrounding resting fluid. The slab is translationally invariant in the $y$ direction and moving in the $x$ direction.

The perturbation is said to be "small" because the initial speed of the slab is small with respect to the ion acoustic speed. In such a case, the mode is linear and we aim in this practical work at calculating the speed at which this small perturbation is propagating and verify that it is the ion acoustic speed.

In [ ]:
import os
import sys

In [ ]:
home = os.environ['HOME']
work_path = os.path.join(home, 'far/farTeach/2026/weak')
src_path = os.path.join(home, 'far/PHARE')
build_path = os.path.join(home, 'far/builds/release/ufunc')

In [ ]:
sys.path.append(os.path.join(src_path, "pyphare"))

In [ ]:
import subprocess
import pyphare
import matplotlib.pyplot as plt
from pyphare.pharesee.run import Run
from pyphare.core.operators import dot, cross, sqrt, modulus, grad
from pyphare.core.ufuncs import gF, peakIds
import numpy as np
from numpy import polyfit

## Run the `PHARE` code in dedicated conditions

In [ ]:
Ti = 0.02
Te = 0.1
run_name = "wpc"

It is suggested to play with both electron and ion temperature. The plasma is magnetized, but looking at the perturbation polarisation, in the hybrid limit ($\omega < \omega_{pe}$), the ion acoustic mode is the only one that can propagate. Hence, for the size of the domain and total integration time, these temperature are controlling :
* the ion acoustic speed. if this one is too small, the mde wont be seable. if it is too large, it will exit the domain (with BC problems)
* the rate of the Landau damping, because the ion acoustic speed has to be close to the ion thermal velocity

As $m_e \to 0$, $v_{te}$ will always be very large compared to $\omega/k$, whatever $T_e$. But if the ion thermal velocity is too small, they wont be able to be Landau damped.

In [ ]:
run_path = os.path.join(work_path, run_name)

In [ ]:
if os.path.isdir(run_path):
    files = os.listdir(run_path)
else:
    files = []

In [ ]:
if 'PYTHONPATH' in os.environ:
    os.environ['PYTHONPATH'] += os.pathsep + os.path.join(src_path, "pyphare")
else:
    os.environ['PYTHONPATH'] = os.pathsep + os.path.join(src_path, "pyphare")
os.environ['PYTHONPATH'] += os.pathsep + build_path

All the path are now set, so that a **PHARE** simulation is ready to run. If the H5 files of  the run doesn't exist, the cell below will then run the code and strore the data using a h5 files in the dir given in the simulation script `wp.py`

In [ ]:
if 'ions_charge_density.h5' not in files :
    os.chdir(work_path)
    # subprocess.run(['/usr/bin/python3', work_path+'/wp.py', run_name, str(Te), str(Ti)], env=os.environ)
    subprocess.run(['mpirun', '-n', '2', '/usr/bin/python3', work_path+'/wp.py', run_name, str(Te), str(Ti)], env=os.environ)

In [ ]:
run  = Run(run_path)

In [ ]:
times = np.asarray((20, 40, 60, 80, 100))

In [ ]:
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [ ]:
time = times[3]

In [ ]:
V = run.GetVi(time)

In [ ]:
fig, ax = plt.subplots(figsize=(6,2))

V.plot(qty='x', ax=ax, ls='solid', lw=1.0, color='tab:blue', ylabel='X-velocity')

The data obtained from the **PHARE** simlulation are too noisy so that the peaks can easily be found. We then create a new patch hierarchy as a deepcopy of the former one, but for which all the dataset are smoothed using a Gaussian filter. This one is then the Gaussian filter of `pyphare`.

In [ ]:
v = gF(V, sigma=6)

We then use a `pyphare` `find_peaks`, called `peaksIds` to find the peaks (using the same syntax and format as the method of `scipy`)

In [ ]:
pks, hgs = peakIds(v, names=['x',], height=0.015)

In [ ]:
fig, ax = plt.subplots(figsize=(6,2))

V.plot(qty='x', ax=ax, ls='solid', lw=1.0, color='tab:blue', ylabel='X-velocity')
v.plot(qty='x', ax=ax, ls='solid', lw=2.0, color='tab:orange', ylabel='X-velocity')
for p in pks:
    ax.axvline(x=p, color='black', linestyle='dotted')

You can then do the job for a set of times to investigate how the small perturbation is propagating

In [ ]:
fig, ax = plt.subplots(figsize=(12,2))

pks_plus = []
pks_minus = []
hgs_plus = []
hgs_minus = []

for i, time in enumerate(times):
    V = run.GetVi(time)
    v = gF(V, sigma=6)
    pks, hgs = peakIds(v, names=['x',], height=0.0135, distance=20)

    pks_minus.append(pks[1])
    pks_plus.append(pks[0])
    hgs_minus.append(hgs[1])
    hgs_plus.append(hgs[0])

    v.plot(qty='x', ax=ax, ls='solid', lw=2.0, color=colors[i], ylabel='X-velocity')

    for p in pks:
        ax.axvline(x=p, color='black', linestyle='dotted')

In [ ]:
fig, ax = plt.subplots(figsize=(8,3))

ax.plot(times, pks_minus, marker="o", color=colors[5], ls="None")
slope_m, origin_m = polyfit(times, pks_minus, 1)
ax.plot(times, times*slope_m+origin_m, color=colors[5])

ax.plot(times, pks_plus, marker="o", color=colors[6], ls="None")
slope_p, origin_p = polyfit(times, pks_plus, 1)
ax.plot(times, times*slope_p+origin_p, color=colors[6])

ax.text(20, 100, 'positive slope : {:.3f}'.format(slope_p))
ax.text(20, 20, 'negative slope : {:.3f}'.format(slope_m))

ax.set_ylim([0, 128])
ax.set_xlabel('Time')
ax.set_ylabel('X-position')

In [ ]:
v_phi = np.mean(np.abs([slope_m, slope_p]))
print('phase velocity of the mode : {:.3f}'.format(v_phi))

The plasma is not magnetized so that the number of linear mode that can exist is limited... What is the number mode that exist ? 

There exist 3 modes :
* the Bohm and Gross mode
* the light mode
* the ion acouistic mode

But the 2 first modes are very high frequency... above the plasma frequency. Hence, in the hybrid frame, only the ion acoustic mode exist... which should then be the one we observe.

In [ ]:
Gamma_e = 1
Gamma_i = 3
print("ion acoustic speed : ", np.sqrt(Gamma_e*Te+Gamma_i*Ti))

In [ ]:
from scipy import odr

def model_fn(p, x):
    a, b, c = p
    return a+b*np.exp(c*x)
model = odr.Model(model_fn)

x = times
model = odr.Model(model_fn)

y = hgs_minus
data = odr.Data(x, y)
odr_obj = odr.ODR(data, model, beta0=[0.01,0.01,0.01])
out_minus = odr_obj.run()
# out_minus.pprint()
# print(out_minus.beta)
b0, b1, b2 = out_minus.beta

y = hgs_plus
odr_obj = odr.ODR(odr.Data(x, y), model, beta0=[0.001,0.001,0.001])
out_plus = odr_obj.run()
# out_plus.pprint()
# print(out_plus.beta)
B0, B1, B2 = out_plus.beta

print("growth rate for the minus side : ", b2)
print("growth rate for the plus side : ", B2)

In [ ]:
fig, ax = plt.subplots(figsize=(8,3))

ax.semilogy(times, hgs_minus, marker="o", color=colors[5], ls="None")
ax.semilogy(times, model_fn(out_minus.beta, times), color=colors[5], ls=":")

ax.semilogy(times, hgs_plus, marker="o", color=colors[6], ls="None")
ax.semilogy(times, model_fn(out_plus.beta, times), color=colors[6], ls=":")

# ax.text(20, 0.028, 'positive growth rate : {:.3f}'.format(B2))
# ax.text(20, 0.027, 'negative growth rate : {:.3f}'.format(b2))

#ax.set_ylim([0.024, 0.036])
ax.set_xlabel('Time')
ax.set_ylabel('Amplitude')

In [ ]:
time=100
r = Run(run_path)
ions = r.GetParticles(time, ["protons",])

fig, ax = plt.subplots(figsize=(10,4))

p,f = ions.dist_plot(axis=("x", "Vx"),
                     ax=ax,
                     norm = 0.4,
                     finest=True,
                     gaussian_filter_sigma=1,
                     vmin=-1,vmax=1,
                     dv=0.01,
                     title="weak perturbation at time : {:.2f}".format(time),
                    )

# FAIRE UN DIST_PLOT AVEC DES POINTS, PAS DES BINS DE COULEUR